In [1]:
import cv2
import numpy as np
import os

In [2]:
def crop_and_save(original_image_path, mask_image_path):
    # Load the original and mask image
    original_image = cv2.imread(original_image_path)
    mask_image = cv2.imread(mask_image_path)

    # Define a dictionary to map color labels in the mask image to their corresponding BGR color values
    label_color_dict = {
                        "leafrust":(0,0,255),
                        "leafminer":(255,0,0),
                        "freefeeder":(100,0,200),
                        "leafskeletonizer":(0,190,255)
                        }

    # Convert the mask image to grayscale
    gray_mask = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)

    # Apply a threshold to convert the grayscale mask image to a binary image
    ret, binary_mask = cv2.threshold(gray_mask, 127, 255, cv2.THRESH_BINARY)

    # Find contours of objects in the binary mask image
    contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop through the contours and identify the color of each contour
    for contour in contours:
        # Find the bounding box of the contour
        x, y, w, h = cv2.boundingRect(contour)

        # Get the color label of the current contour from the mask image
        color_label = mask_image[y + int(h / 2), x + int(w / 2)]

        label = None
        for key, value in label_color_dict.items():
            if np.array_equal(value, color_label):
                label = key
                break
        if label is None:
          continue

        # Create a directory with the name of the color label if it doesn't exist
        if not os.path.exists(label):
            os.makedirs(label)

        # Crop the corresponding region from the original image
        cropped_image = original_image[y:y+h, x:x+w]

        # Save the cropped image in the directory for the corresponding color label
        file_name = os.path.join(label, f"{label}_{len(os.listdir(label))+1}.jpg")
        cv2.imwrite(file_name, cropped_image)